# Lecture 12 - Sampling Models

In [ ]:
using DifferentialEquations, ParameterizedFunctions, Distributions, Random, Plots; gr();

## Mass-Spring Model

In [ ]:
MS_rhs = @ode_def begin
    dx = y
    dy = -(k/m)*x - (g/m)*y
end m k g;

In [ ]:
function solve_MS(m,k,g,u0,u1)
    u0 = [u0, u1]
    tspan = (0.0, 50000.0)
    p = [m, k, g]
    prob = ODEProblem(MS_rhs, u0, tspan, p)
    sol = solve(prob)
    return sol
end;

In [ ]:
solve_MS(10,0.5,0.1,4.0,0.0)
@time sol = solve_MS(10,0.5,0.1,4.0,0.0);

In [ ]:
function eqtime_MS(m,k,g,u0,u1)
    sol = solve_MS(m,k,g,u0,u1)
    threshold = 0.00001
    eqtime = findfirst([sum(y) for y in [abs.(x) for x in sol.u]] .< threshold)
    return sol.t[eqtime]
end;

In [ ]:
eqtime_MS(10,0.1,0.1,4.0,0.0)
@time eqtime_MS(10,0.1,0.1,4.0,0.0);

## SIR Model

In [ ]:
SIR_rhs = @ode_def begin
    ds = - c * s * i
    di = c * s * i - h * i
    dr = h * i
end c h;

In [ ]:
function solve_SIR(c,h)
    u0 = [0.99999 0.000001 0.0]
    tspan = (0, 5000.0)
    p = [c , h]
    prob = ODEProblem(SIR_rhs, u0, tspan, p)
    sol = solve(prob)
    return sol
end;

In [ ]:
solve_SIR(0.5,0.2)
@time sol = solve_SIR(0.5,0.1);

In [ ]:
function eqpts_SIR(c,h)
    sol = solve_SIR(c,h) 
    return sol.u[end]
end;

In [ ]:
eqpts_SIR(0.5,0.1)
@time eqpts_SIR(0.5,0.2);

## Sobol G Function

Defined as

$$ f(\vec{x}) = \prod_{i=1}^N \frac{|4x_i - 2| + a_i}{1-a_i} $$

where

$$ \vec{x} = [x_1, x_2, \cdots, x_N] , \quad x_i \sim \mathbf{U}(0,1)$$

and

$$ a_i = \frac{i - 2}{2} \text{ for all } i = 1, \cdots, N .$$

In [ ]:
function solve_SB(x; a=[0, 0.25, 99.0])
    if length(x) != length(a)
        error("Vectors must be the same dimension")
    else
        y = prod([(abs(4*x[i] - 2) + a[i])./(1+a[i]) for i = 1:length(x)])
    end
    return y
end;

In [ ]:
solve_SB([0.1 0.2 0.3])
@time sol = solve_SB([0.1 0.2 0.4]);